In [13]:
import pandas as pd
import sklearn
import numpy as np
from sklearn import preprocessing

In [14]:
fl = pd.read_csv("movie_data.csv")

In [15]:
#left actor1,actor2,actor3 and director likes,facenumber_in_poster,aspect ratio
#left names of cast too mant levels need to see what is to be done
continuous_vars = [
    'num_critic_for_reviews','gross','duration','num_voted_users','actor_1_facebook_likes',
    'actor_2_facebook_likes','actor_3_facebook_likes','director_facebook_likes',
    'num_user_for_reviews','budget','title_year','movie_facebook_likes'
    ]

def preprocess(fl):
    for var in continuous_vars:
        if var == 'title_year':
            fl[var].fillna(fl[var].mode()[0], inplace = 'True')
        else:
            fl[var].fillna(fl[var].mean(), inplace='True')
        for index,row in fl.iterrows():
            if row['num_critic_for_reviews']>(140 + 121*3):
                val =  140
                fl.set_value(index,'num_critic_for_reviews',val)
            if row['gross']>(47900000 + 68230000*3):
                val =  47900000
                fl.set_value(index,'gross',val)
            if row['num_voted_users']>(83591 + 138708*3):
                val =  83591
                fl.set_value(index,'num_voted_users',val)
            if row['actor_1_facebook_likes']>(6583 + 15713*3):
                val =  6583
                fl.set_value(index,'actor_1_facebook_likes',val)
            if row['actor_2_facebook_likes']>(1643 + 4124*3):
                val =  1643
                fl.set_value(index,'actor_2_facebook_likes',val)
            if row['actor_3_facebook_likes']>(643 + 1689*3):
                val =  643
                fl.set_value(index,'actor_3_facebook_likes',val)
            if row['director_facebook_likes']>(681 + 2809*3):
                val =  681
                fl.set_value(index,'director_facebook_likes',val)
            if row['num_user_for_reviews']>(271 + 373*3):
                val =  271
                fl.set_value(index,'num_user_for_reviews',val)
            if row['budget']>(40612860 + 222575500*3):
                val =  40612860
                fl.set_value(index,'budget',val)
            if row['duration']>(107 + 25*3):
                val =  107
                fl.set_value(index,'budget',val)
            if row['movie_facebook_likes']>(7638 + 19805*3):
                val =  7638
                fl.set_value(index,'movie_facebook_likes',val)

#def normalize(fl):
    #if some other model than rf has to be used
                    
preprocess(fl)        


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
list_genres = []
for index,row in fl.iterrows():
    genre = (" ").join(row['genres'].split('|'))
    list_genres.append(genre)
count_vectorizer.fit_transform(list_genres) #for test data also   
categorical_vars = ['content_rating','language']
text_vars = ['genres','plot_keywords']

In [17]:
print("Vocabulary:", count_vectorizer.vocabulary_)
freq_term_matrix = count_vectorizer.transform(list_genres)
dt = freq_term_matrix.todense()
import operator
sorted_cols = sorted(count_vectorizer.vocabulary_.items(), key=operator.itemgetter(1))
cols= []
for col in sorted_cols:
    cols.append(col[0])
df_genre_words = pd.DataFrame(data=dt[0:,0:],columns= cols)

Vocabulary: {'tv': 27, 'romance': 21, 'documentary': 6, 'show': 24, 'drama': 7, 'crime': 5, 'comedy': 4, 'mystery': 17, 'film': 11, 'animation': 2, 'action': 0, 'family': 8, 'fantasy': 9, 'noir': 19, 'fi': 10, 'music': 15, 'reality': 20, 'game': 12, 'news': 18, 'war': 28, 'sport': 25, 'adventure': 1, 'western': 29, 'history': 13, 'sci': 22, 'biography': 3, 'musical': 16, 'thriller': 26, 'horror': 14, 'short': 23}


In [18]:
"""
count_vectorizer1 = CountVectorizer()
list_pk = []
for index,row in fl.iterrows():
    pk = (" ").join(str(row['plot_keywords']).split('|'))
    list_pk.append(pk)
count_vectorizer1.fit_transform(list_pk) #for test data also   
categorical_vars = ['country','content_rating']
text_vars = ['genres','plot_keywords']
print("Vocabulary:", count_vectorizer1.vocabulary_)
freq_term_matrix = count_vectorizer.transform(list_pk)
print(freq_term_matrix.todense().shape)
"""

'\ncount_vectorizer1 = CountVectorizer()\nlist_pk = []\nfor index,row in fl.iterrows():\n    pk = (" ").join(str(row[\'plot_keywords\']).split(\'|\'))\n    list_pk.append(pk)\ncount_vectorizer1.fit_transform(list_pk) #for test data also   \ncategorical_vars = [\'country\',\'content_rating\']\ntext_vars = [\'genres\',\'plot_keywords\']\nprint("Vocabulary:", count_vectorizer1.vocabulary_)\nfreq_term_matrix = count_vectorizer.transform(list_pk)\nprint(freq_term_matrix.todense().shape)\n'

In [19]:
#handle categoricals
#not using langauge as low importance
def preprocess(fl):
    for feature in categorical_vars:
        dummies = pd.get_dummies(fl[feature])
        fl= pd.concat([fl, dummies], axis= 1 )
        fl = fl.drop(feature, 1)
    return fl
fl = preprocess(fl)

In [20]:
#merge all the df's and remove note to be used vars
df_train= pd.concat([fl , df_genre_words], axis= 1 )
df_train = df_train.drop(['color','director_name','actor_2_name','actor_3_name','plot_keywords','country'
                         ,'aspect_ratio','actor_1_name','genres','facenumber_in_poster','movie_imdb_link'],axis=1)

In [21]:
df_train.to_csv('imdb_processed.csv')